# Homework 5 RF Accuracy Improvement

This assignment is inspired by examples of Shan-Hung Wu from National Tsing Hua University.

Requirement: improve the accuracy per feature of the following code from 0.03 up to at least 0.45 and accuracy should be more than 0.92

Here are three hints:

    You can improve the ratio by picking out or "creating" several features.
    Tune hyperparameters
    The ratio can be improved from 0.03 up to 0.47.

In [29]:
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.inspection import permutation_importance
from scipy.stats import uniform
import numpy as np

# load the breast_cancer dataset
init_data = load_breast_cancer()
(X, y) = load_breast_cancer(return_X_y=True)

k = 2

rf_classifier = RandomForestClassifier()
rf_classifier.fit(X,y)
result = permutation_importance(rf_classifier, X, y, n_repeats=30, random_state=42)
importances = result.importances_mean
top_k_indices = np.argsort(importances)[-k:]
X = X[:, top_k_indices]

param_dist = {
    'n_estimators': [10,50,100,200,300],
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2', None],
    'bootstrap': [True, False],
    'class_weight': [None, 'balanced', 'balanced_subsample']
}

randm_src = RandomizedSearchCV(
    rf_classifier,     
    param_distributions=param_dist,
    cv=5
)
randm_src.fit(X, y)

print(randm_src.best_params_)
print(randm_src.best_score_)

cv_scores = cross_val_score(RandomForestClassifier(**randm_src.best_params_), X, y, cv=5)

average_accuracy = np.mean(cv_scores)

average_accuracy_per_feature = average_accuracy / X.shape[1]

print("Average Accuracy: {:.2f}%".format(average_accuracy * 100))
print("Average Accuracy per Feature: {:.2f}%".format(average_accuracy_per_feature * 100))


{'n_estimators': 10, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': None, 'criterion': 'gini', 'class_weight': 'balanced', 'bootstrap': True}
0.9261450085390468
Average Accuracy: 92.09%
Average Accuracy per Feature: 46.04%


C:\Users\Lucas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
25 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Lucas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Lucas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-pack